# Background

This notebook provides examples to accompany the C3DIS 2017 _Exploring Python Decorators_ presentation. Equally, it can be used as a stand-alone exploration of decorators.

These examples have been tested with Python 3.5 and 3.6 but should also work with Python 2.7

Working through this notebook will:
* get you started with writing decorators;
* provide some examples of use and references;
* as well as pointing out some problems to be aware of.

# Topics covered
* Toward decorators
* Timer decorator using a function
* NOP decorator and its uses
* Timer decorator using a class
* Memoising decorator (composition and side effects)
* Simple type checking
  * decorator parameters
  * type annotations
* Timing (is) everything; decorating a class
* Some other uses for decorators

# Toward decorators
Suppose we want to time a function's execution...

In [ ]:
def range_sum(n):
    return sum(range(1, n+1))

In [ ]:
import time

t0 = time.clock()
print(range_sum(10000000))
t1 = time.clock()

print("Elapsed time: {0:1.5f} seconds".format(t1-t0))

But there are probably other functions to time, so we could write a function like this...

In [ ]:
import time


def timer(func, *args):
    t0 = time.clock()
    result = func(*args)
    t1 = time.clock()
    print("Elapsed time: {0:1.5f} seconds".format(t1-t0))
    return result

and *try* calling it...

In [ ]:
timer(range_sum(10000000))

Not quite.

We've passed in the integer result of calling `range_sum` to the `timer` function.

Second attempt...

In [ ]:
args = [10000000]
timer(range_sum, *args)

...or just...

In [ ]:
timer(range_sum, *[10000000])

This _works_ but it's a bit cumbersome.

If we knew there was only ever one argument, we could define the `timer` function to take just one and have:

In [ ]:
timer(range_sum, 10000000)

But, we want to generalise `timer` to handle any kind of function and do so without changing how we would normally call a function.

Next attempt: a function that returns a function...

# Timer decorator using a function

In [ ]:
import time

def timer(func):
    # For arbitrary function calls, we include positional and keyword args
    def wrapped(*args, **kwargs):
        t0 = time.clock()
        result = func(*args, **kwargs)
        t1 = time.clock()
        print("Elapsed time: {0:1.5f} seconds".format(t1-t0))
        return result
    return wrapped

In [ ]:
timer(range_sum)

In [ ]:
timer(range_sum)(10000000)

This works by returning a function (`wrapped` or `inner` or `laundromat`) that can be applied to a normal parameter list.

Almost there!

This higher-order function has _just the right form_ necessary for a decorator.

Since `func` is passed as a parameter to `timer`, it becomes part of `wrapped`'s environment, so is remembered by it (i.e. `wrapped` is a [closure](https://stackoverflow.com/questions/2796855/python-closures-example-code)).

So, now we can decorate functions of interest and call them normally e.g.

In [ ]:
@timer
def range_sum(n):
    return sum(range(1,n+1))

@timer
def iterative_sum(n):
    sum = 0
    for k in range(1,n+1):
        sum += k
    return sum

@timer
def gaussian_sum(n):
    return n*(n+1)/2

But what does a function decorated with timer look like?

In [ ]:
range_sum

Just a function that will call another function.

So, now we can just call the (decorated) functions...

In [ ]:
iterative_sum(10000000)
range_sum(10000000)
gaussian_sum(10000000)

So, in this way, the decorator annotation is syntactic sugar.

# NOP decorator

Before continuing, let's look at the simplest decorator (and why it's of interest).

In [ ]:
def nop(func):
    def inner(*args, **kwargs):
        pass
    return inner

In [ ]:
def f():
    print("Hello, decorator")

f()

In [ ]:
@nop
def g():
    print("Hello, decorator (or not)")

g()

The [Python 3 unittest library](https://docs.python.org/3/library/unittest.html#skipping-tests-and-expected-failures) is an example of how this can be useful as in the next cell.

In [ ]:
import unittest
import requests

class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(requests.__version__ < '2.13.0',
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

# Timer decorator using a class
Sometimes, a class may be more suitable when defining a decorator. We can redefine our `timer` function like this.

The function is passed to `__init__` and the arguments to `__call__`. It's used in the same way as a decorator.

In [ ]:
import time

class timer:
    def __init__(self, func):
        print(">timer init")
        self.func = func

    def __call__(self, *args, **kwargs):
        print(">timer entry")
        t0 = time.clock()
        result = self.func(*args, **kwargs)
        t1 = time.clock()
        print("Elapsed time: {0:1.5f} seconds".format(t1-t0))
        print("<timer exit")
        return result

 We don't gain anything by using a class for `timer` but we'll see examples of where it can be useful.
 
 The reason for the additional print statements will be clear in the next example.

# Memoising decorator (and composing decorators)
Suppose you had a function that took a while to compute its result so that caching results is useful.

In [ ]:
class memoize:
    def __init__(self, func):
        print(">memoize init")
        self.func = func
        self.cache = {}

    def __call__(self, *args):
        print(">memoize entry")
        if args not in self.cache:
            self.cache[args] = self.func(*args)
        else:
            print("result for {0} was cached".format(*args))
        print("<memoize exit")
        return self.cache[args]

In [ ]:
@timer
@memoize
def iterative_sum(n):
    print(">iterative_sum entry")
    sum = 0
    for i in range(1,n+1):
        sum += i
    print("<iterative_sum exit")
    return sum


@timer
@memoize
def gaussian_sum(n):
    return n*(n+1)/2

Notice that for each decorated function, an instance of the decorator classes is _created_ in a particular order: the reverse of their lexical appearance.

When called, the decorators and function are executed in the order of lexical appearance in the code, in this case:
1. timer
2. memoize
3. iterative_sum

In [ ]:
iterative_sum(10000000)

Now we make the same call as before. This time the value is remembered, not re-computed.

In [ ]:
iterative_sum(10000000)

Of course, since `timer` calls `memoize` which calls `iterative_sum`, the execution time includes `memoize`'s execution time too (even though small).

Changing decoration order has consequences just as changing the order in a function composition will, e.g. f(g(x)) vs g(f(x)).

After the first call to `iterative_sum` with a particular value, a lexical reversal of `@timer` and `@memoize` will lead to the cached value being returned by `memoize` but the timer decorator will not print an "elapsed time" message since with the reversal, `memoize` is invoked first and never needs to call anything further downstream after caching a value.

So, depending upon decoration and order and implementation, there can be "interesting" side effects.

In [ ]:
@memoize
@timer
def iterative_sum(n):
    print(">iterative_sum entry")
    sum = 0
    for i in range(1,n+1):
        sum += i
    print("<iterative_sum exit")
    return sum

In [ ]:
iterative_sum(10000000)

In [ ]:
iterative_sum(10000000)

For more decorator call sequence examples, see the _Exploring Decorators Supplement_ notebook. 

# Simple type checking (decorators with parameters)
Decorators can take parameters.

Suppose you wanted to impose simple constraints on a function's types. 

In this simple code, you would expect the third call to `pow2` to fail...

In [ ]:
def pow2(n):
    return 2**n

print(pow2(10))
print(pow2(20.5))
print(pow2(2, 10))

...but what if you also wanted to impose constraints on permissible types for `n`?

In [ ]:
class check:
    def __init__(self, *args):
        self.types = args

    def __call__(self, func):
        def wrapped(*args, **kwargs):
            length_check(func.__name__, self.types, args)
            type_check(func.__name__, self.types, args)
            return func(*args, **kwargs)


        def length_check(func_name, formals, actuals):
            # Do the formal and actual lists have the same length?
            if len(formals) != len(actuals):
                msg = '{0}: argument count mismatch: ' \
                      'formals: {1}, actuals: {2}'. \
                          format(func_name, len(formals), len(actuals))
                raise TypeError(msg)


        def type_check(func_name, formals, actuals):
                # Check each argument's type.
                for index, expected_type in enumerate(formals):
                    actual_type = type(actuals[index])
                    if actual_type != expected_type:
                        msg = '{0}: argument {1} is {2} but {3} expected'. \
                                  format(func_name, index+1, 
                                         actual_type.__name__, 
                                         expected_type.__name__)
                        raise TypeError(msg)

        return wrapped

The decorator mechanism here requires that we make a distinction between decorator arguments (passed to `__init__`) and function arguments. So, the `__call__` method now takes the function to be decorated and returns a decorated function which takes the ultimate function arguments of interest.

We can use this to check that we only ever pass an integer to `pow2`, for example:

In [ ]:
@check(int)
def pow2(n):
    return 2**n

print(pow2(10))
print(pow2(20.5))
print(pow2(2, 10))

Instead, use type hints/annotation (from Python 3.5):

In [ ]:
def pow2(n: int):
    return 2**n

and a tool like `mypy` or a type checking decorator:
* http://www.machinalis.com/blog/a-day-with-mypy-part-1
* https://pypi.python.org/pypi/typecheck-decorator

Or, we could rewrite our `@check` to use type annotations...

In [ ]:
class check:
    def __init__(self, func):
        self.func = func
        self.annotations = func.__annotations__

    def __call__(self, *args, **kwargs):
        self.length_check(self.annotations, args)
        self.type_check(self.annotations, args)
        return self.func(*args, **kwargs)
    
    
    def length_check(self, formals, actuals):
        if len(formals) != len(actuals):
            msg = '{0}: argument count mismatch: ' \
                  'formals: {1}, actuals: {2}'. \
                      format(self.func.__name__, 
                             len(formals), len(actuals))
            raise TypeError(msg)

    def type_check(self, formals, actuals):
        for formal, actual in zip(formals, actuals):
            actual_type = type(actual)
            expected_type = formals[formal]
            if actual_type != expected_type:
                msg = '{0}: {1} is {2} but {3} expected'. \
                          format(self.func.__name__, formal,
                                 actual_type.__name__, 
                                 expected_type.__name__)
                raise TypeError(msg)

...and then this...

In [ ]:
@check
def pow2(n: int):
    return 2**n

print(pow2(10))
print(pow2(20.5))
print(pow2(2, 10))

Both implementations need to be generalised, e.g. handling return types.

# Timing (is) everything; decorating a class
What if we want to decorate some or all methods in a class, e.g. to time them?

I'm glad you asked...

Given our old `timer` decorator friend (with a line to print the name of the function for additional output):

In [ ]:
import time

class timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(self.func.__name__)
        t0 = time.clock()
        result = self.func(*args, **kwargs)
        t1 = time.clock()
        print("Elapsed time: {0:1.5f} seconds".format(t1-t0))
        return result

It would be nice to be able to decorate methods in a class, like this:

In [ ]:
@class_decorator(timer, "iterative_sum", "gaussian_sum")
class Summers:
    def iterative_sum(self, n):
        sum = 0
        for i in range(1, n+1):
            sum += i
        return sum

    def range_sum(self, n):
        return sum(range(1, n+1))

    def gaussian_sum(self, n):
        return n*(n+1)/2

...and then say:

In [ ]:
summers = Summers()

print(summers.iterative_sum(10000000))
print(summers.range_sum(10000000))
print(summers.gaussian_sum(10000000))

The missing piece is `class_decorator`, a function that takes a decorator and method names and returns a `class_rebuilder` function, just like our first `timer` function did.

That wrapped function returns a class  (`DecoratedClass`) that specialises a target class (`Summers` in our case) by decorating the specified methods (supplied in `method_names`), by overriding the `__get_attribute__()` method. Notice that `obj` is a function object that is passed to the decorator in the just the same way as we have been doing until now.

In [ ]:
def class_decorator(decorator, *method_names):
    def class_rebuilder(cls):
        class DecoratedClass(cls):
            def __getattribute__(self, attr_name):
                obj = super().__getattribute__(attr_name)
                if hasattr(obj, '__call__') and attr_name in method_names:
                    return decorator(obj)
                return obj
        return DecoratedClass
    return class_rebuilder

Thanks to Ben Leighton for the question (last time I gave a [talk about decorators](https://confluence.csiro.au/display/BioinformaticsCore/Bioinformatics+Seminar+Series+2017)) and the pointer to https://andrefsp.wordpress.com/2012/08/23/writing-a-class-decorator-in-python.

I've generalised the code from that page to allow any decorator to be applied to class methods.

# Some other uses for decorators
  * Frameworks, e.g. `@route` in Flask web framework
  * Execution tracing
  * Logging
  * Retries
  * Authentication
  * Concurrency, synchronisation (e.g. **monitors**)
  * Periodic timers
  * Method decorators
    * `@property`
    * `@staticmethod`
    * `@classmethod`
  * UIs (**@gooey** needs to be made compatible with Python 3)
  * **Native code generation and execution, i.e. JIT (ala Numba)**
  * SLURM batch job submission and checking (Daniel Collins mentioned seeing this used recently)
  
  The items in bold relate to ideas I plan to follow up on or have already done some work towards.

# Conclusions
The decorator mechanism and syntax in Python is fairly simple and easy to get started with, but when multiple decorators are applied to the same function, side effects must be considered.

The possible applications of this simple mechanism are numerous.

# References

### Decorators
* https://www.python.org/dev/peps/pep-0318/
* https://realpython.com/blog/python/primer-on-python-decorators/
* https://wiki.python.org/moin/PythonDecorators
* https://wiki.python.org/moin/PythonDecoratorLibrary 
* https://andrefsp.wordpress.com/2012/08/23/writing-a-class-decorator-in-python/
* https://www.codementor.io/sheena/advanced-use-python-decorators-class-function-du107nxsv
* https://docs.python.org/3/library/unittest.html#skipping-tests-and-expected-failures
* http://www.artima.com/weblogs/viewpost.jsp?thread=240808#function-decorators
* https://stackoverflow.com/questions/2366713/can-a-python-decorator-of-an-instance-method-access-the-class
* https://www.oreilly.com/ideas/5-reasons-you-need-to-learn-to-write-python-decorators
* https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work

### Supporting Information
* https://stackoverflow.com/questions/2796855/python-closures-example-code
* https://stackoverflow.com/questions/3394835/args-and-kwargs
* https://en.wikipedia.org/wiki/Python_syntax_and_semantics 
* https://stackoverflow.com/questions/9450656/positional-argument-v-s-keyword-argument
* http://numba.pydata.org/
* https://docs.python.org/3/library/functools.html
* https://stackoverflow.com/questions/13591970/does-python-optimize-tail-recursion